Quantitative Momentum Strategy
"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stock

Library Imports

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module


Importing Our List of Stocks

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Making Our First API Call

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


In [6]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2852577670570,
 'week52high': 189.65,
 'week52low': 134.39,
 'week52highSplitAdjustOnly': 190.7,
 'week52lowSplitAdjustOnly': 132.9,
 'week52change': 0.16548684393077284,
 'sharesOutstanding': 16782851320,
 'float': 0,
 'avg10Volume': 65577710,
 'avg30Volume': 69689055,
 'day200MovingAvg': 158.81,
 'day50MovingAvg': 163.23,
 'employees': 147530,
 'ttmEPS': 6.34,
 'ttmDividendRate': 0.9020696244866108,
 'dividendYield': 0.005385413084534317,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-31',
 'nextEarningsDate': '2022-10-19',
 'peRatio': 28.35405577098791,
 'beta': 1.311534172649077,
 'maxChangePercent': 66.51236062620619,
 'year5ChangePercent': 3.6633031081649063,
 'year2ChangePercent': 0.4872981171442672,
 'year1ChangePercent': 0.16926853651009618,
 'ytdChangePercent': -0.031110365807955884,
 'month6ChangePercent': 0.02894469280567878,
 'month3ChangePercent': 0.2529380077089108,
 'month1ChangePercent': 0.12270612679891485,
 'day30Chang

Parsing Our API Call

In [7]:
data['year1ChangePercent']

0.16926853651009618

Executing A Batch API Call & Building Our DataFrame

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [10]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,143.26,-0.180012,N/A
1,AAL,14.2,-0.242969,N/A
2,AAP,207.77,0.012522,N/A
3,AAPL,177.23,0.167547,N/A
4,ABBV,146.43,0.246172,N/A
...,...,...,...,...
130,DG,254.87,0.092697,N/A
131,DGX,139.41,-0.1023,N/A
132,DHI,76.3,-0.210466,N/A
133,DHR,304.74,-0.090579,N/A


Removing low momentum stocks

In [11]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CF,106.6,1.430896,N/A
1,APA,36.55,1.32034,N/A
2,COP,108.45,1.094134,N/A
3,CVX,164.05,0.757079,N/A
4,CNC,99.4,0.527118,N/A
5,ADM,89.74,0.522316,N/A
6,CTVA,64.05,0.487715,N/A
7,CI,306.18,0.449013,N/A
8,AZO,2423.81,0.429894,N/A
9,ANET,136,0.41854,N/A


Calculating the Number of Shares to Buy

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

That's not a number! 
 Try again:
100000000000


In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CF,106.6,1.430896,18393849
1,APA,36.55,1.32034,53646629
2,COP,108.45,1.094134,18080076
3,CVX,164.05,0.757079,11952357
4,CNC,99.4,0.527118,19726200
5,ADM,89.74,0.522316,21849613
6,CTVA,64.05,0.487715,30613338
7,CI,306.18,0.449013,6404024
8,AZO,2423.81,0.429894,808967
9,ANET,136,0.41854,14417531


Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

1-month price returns
3-month price returns
6-month price returns
1-year price returns
Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [ ]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        


In [16]:
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

Calculating Momentum Percentiles
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

One-Year Price Return
Six-Month Price Return
Three-Month Price Return
One-Month Price Return

In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

In [17]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,141.08,N/A,-0.178052,N/A,0.04638,N/A,0.106569,N/A,0.155523,N/A,N/A
1,AAL,14.56,N/A,-0.238797,N/A,-0.214265,N/A,-0.131247,N/A,-0.068613,N/A,N/A
2,AAP,208.72,N/A,0.012566,N/A,-0.039125,N/A,0.151997,N/A,0.07489,N/A,N/A
3,AAPL,172.83,N/A,0.170013,N/A,0.028504,N/A,0.251108,N/A,0.127741,N/A,N/A
4,ABBV,144.01,N/A,0.246455,N/A,0.002565,N/A,-0.054244,N/A,-0.040699,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,257.19,N/A,0.092361,N/A,0.287772,N/A,0.363508,N/A,0.031458,N/A,N/A
131,DGX,140.53,N/A,-0.103039,N/A,0.037065,N/A,-0.040787,N/A,0.004814,N/A,N/A
132,DHI,74.71,N/A,-0.218612,N/A,-0.104691,N/A,0.087235,N/A,0.02,N/A,N/A
133,DHR,299.27,N/A,-0.091272,N/A,0.098881,N/A,0.163196,N/A,0.148163,N/A,N/A


Calculating the HQM Score
We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

Selecting the 50 Best Momentum Stocks
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [21]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

In [24]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,141.08,1389838,-0.178052,N/A,0.04638,N/A,0.106569,N/A,0.155523,N/A,N/A
1,AAL,14.56,13466925,-0.238797,N/A,-0.214265,N/A,-0.131247,N/A,-0.068613,N/A,N/A
2,AAP,208.72,939432,0.012566,N/A,-0.039125,N/A,0.151997,N/A,0.07489,N/A,N/A
3,AAPL,172.83,1134516,0.170013,N/A,0.028504,N/A,0.251108,N/A,0.127741,N/A,N/A
4,ABBV,144.01,1361561,0.246455,N/A,0.002565,N/A,-0.054244,N/A,-0.040699,N/A,N/A
5,ABC,153.51,1277300,0.296115,N/A,0.085088,N/A,0.015535,N/A,0.081251,N/A,N/A
6,ABMD,277.43,706767,-0.231212,N/A,-0.095321,N/A,0.152506,N/A,-0.019991,N/A,N/A
7,ABT,112.2,1747579,-0.11573,N/A,-0.049723,N/A,-0.024859,N/A,0.01766,N/A,N/A
8,ACN,318.94,614781,-0.04105,N/A,-0.012288,N/A,0.147763,N/A,0.124396,N/A,N/A
9,ADBE,442.491,443124,-0.351878,N/A,-0.040871,N/A,0.067125,N/A,0.060275,N/A,N/A


Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [25]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

String format for tickers
$XX.XX format for stock prices
$XX,XXX format for market capitalization
Integer format for the number of shares to purchase
Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()